In [4]:
import yaml
import os
import numpy as np
import copy

In [10]:
base_config={
                   #'model_name': ,
                   #'model_path': ,
                   #'device': ,
                   'class_groups': [[2*i,2*i+1] for i in range(5)],
                   'dataset_name':'MNIST' ,
                   #'dataset_type':,
                   #'data_root':,
                   'batch_size': 32,
                   #'save_dir': ,
                   'validation_size': 2000,
                   'accuracy': True,
                   'num_batches_per_file':10,
                   'start_file': 0,
                   'num_files':100
}


In [11]:
def create_config_local(config, config_name, C_values=None):
    config['data_root'] = '/home/fe/yolcu/Documents/Code/Datasets/'
    config['device'] = 'cuda'
    config['model_path']=f"/home/fe/yolcu/Documents/Code/THESIS/checkpoints/{config['dataset_name']}/{config['dataset_type']}/{config['model_name']}_{config['dataset_type']}/{config['dataset_name']}_{config['model_name']}"
    config['save_dir'] = f"/home/fe/yolcu/Documents/Code/THESIS/explanations/{config['dataset_name']}/{config['dataset_type']}/{config['model_name']}_{config['dataset_type']}/{config['xai_method']}"
    path = f"local/dualview/{config['dataset_name']}"
    os.makedirs(path, exist_ok=True)
    if C_values is not None:
        base_save_dir=config['save_dir']
        for c in C_values:
            config['save_dir']=f"{base_save_dir}_{c}"
            config['C']=c
            with open(f"{path}/{config_name}_{c}.yaml", 'w') as outfile:
                yaml.dump(config, outfile, default_flow_style=False)
            
    else:
        with open(f"{path}/{config_name}.yaml", 'w') as outfile:
            yaml.dump(config, outfile, default_flow_style=False)
        
def create_config_cluster(config, config_name, C_values=None):
    config['device']='cuda'
    config['data_root'] = '/mnt/dataset/'
    config['model_path']=f"/mnt/checkpoints/{config['dataset_name']}/{config['dataset_type']}/{config['model_name']}_{config['dataset_type']}/{config['dataset_name']}_{config['model_name']}"
    config['save_dir'] = '/mnt/outputs/'
    
    path = f"cluster/dualview/{config['dataset_name']}"
    os.makedirs(path, exist_ok=True)
    if C_values is not None:
        for c in C_values:
            with open(f"{path}/{config_name}_{c}.yaml", 'w') as outfile:
                yaml.dump(config, outfile, default_flow_style=False)   
    else:
        with open(f"{path}/{config_name}.yaml", 'w') as outfile:
            yaml.dump(config, outfile, default_flow_style=False)

In [12]:
#model_names=['basic_conv']
model_names=['basic_conv']
dataset_types=['std', 'group', 'corrupt', 'mark']

batch_sizes_by_method={
        'similarity': 32,
        'influence': 16,
        'tracin':16,
        'mcsvm': 128,
        'representer': 128
}

In [14]:
xai_methods=['mcsvm']
#C_values=[10e-7,10e-5,0.001,0.01, 0.1, 1., 10.]
C_values=[]
base_config['num_files']=500
for name in model_names:
    base_config['model_name']=name
    for dstype in dataset_types:
        if dstype=='group':
            base_config['num_classes']=len(base_config['class_groups'])
        else:
            base_config['num_classes']=10
        base_config['dataset_type']=dstype
        for xai_method in xai_methods:
            base_config['xai_method']=xai_method
            base_config['batch_size']=batch_sizes_by_method[xai_method]
            if xai_method=='mcsvm' and len(C_values)>0:
                #create_config_cluster(base_config, f"{name}_{dstype}_{xai_method}",C_values)        
                create_config_local(base_config, f"{name}_{dstype}_{xai_method}", C_values)
            else:
                create_config_cluster(base_config, f"{name}_{dstype}_{xai_method}")        
                create_config_local(base_config, f"{name}_{dstype}_{xai_method}")

In [5]:
xai_methods=['representer']#, 'influence']
base_config['num_files']=2
base_config['num_batches_per_file']=1
num_jobs=1
file_nums=[i*base_config['num_files'] for i in range(num_jobs)]
for name in model_names:
    base_config['model_name']=name
    for dstype in dataset_types:
        if dstype=='group':
            base_config['num_classes']=len(base_config['class_groups'])
        else:
            base_config['num_classes']=10
        base_config['dataset_type']=dstype
        for xai_method in xai_methods:
            base_config['xai_method']=xai_method
            base_config['batch_size']=batch_sizes_by_method[xai_method]
            for index,file_num in enumerate(file_nums):
                base_config["start_file"]=file_num
                create_config_cluster(base_config, f"{str(index).zfill(len(str(num_jobs)))}_{name}_{dstype}_{xai_method}")
                #create_config_local(base_config, f"{name}_{dstype}_{xai_method}_{str(index).zfill(2)}")

In [6]:
len(str(num_jobs
       ))

3